In [1]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 5

In [4]:
with open("/workspaces/sefik/fer2013/fer2013.csv") as f:
    content = f.readlines()

In [5]:
lines = np.array(content)

In [6]:
num_of_instances = lines.size
print("number of instances: ",num_of_instances)
print("instance length: ",len(lines[1].split(",")[1].split(" ")))

number of instances:  35888
instance length:  2304


In [7]:
x_train, y_train, x_test, y_test = [], [], [], []

In [8]:
for i in range(1,num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")
          
        val = img.split(" ")
            
        pixels = np.array(val, 'float32')
        
        #emotion = keras.utils.to_categorical(emotion, num_classes)
        
        #print(emotion)
    
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except Exception as ex:
        print(ex)

In [9]:
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

28709 train samples
3589 test samples


In [26]:
x_train[0].shape

(48, 48, 1)

In [10]:
import autokeras as ak

In [11]:
model = ak.ImageClassifier(path="/workspaces/sefik/automodels/", verbose=True)

Saving Directory: /workspaces/sefik/automodels/


In [12]:
x_train.shape

(28709, 48, 48, 1)

In [13]:
y_train.shape

(28709,)

In [14]:
model.fit(x_train, y_train, time_limit=60*60*24)

Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   5.586169672012329    |  0.48439999999999994   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+
                                                                      



+----------------------------------------------+
|              Training model 12               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           12           |   3.8213101387023927   |   0.6524000000000001   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|              Training model 13               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+----------------



+----------------------------------------------+
|              Training model 24               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           24           |   3.7875988364219664   |         0.658          |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|              Training model 25               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+----------------



+----------------------------------------------+
|              Training model 36               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           36           |   3.748728704452515    |         0.6588         |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|              Training model 37               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+----------------

In [15]:
model.final_fit(x_train, y_train, x_test, y_test, retrain=True)

                                                                                                    
No loss decrease after 30 epochs.



In [16]:
score = model.evaluate(x_test, y_test)

In [17]:
score

0.6623014767344664

In [18]:
predictions = model.predict(x_test)

In [28]:
instances = 0
correct_ones = 0

for i in range(0, len(predictions)):
    if i < 5:
        print("Prediction: ",predictions[i],", Actual: ",y_test[i])
    
    if predictions[i] == y_test[i]:
        correct_ones = correct_ones + 1
    
    instances = instances + 1

Prediction:  2.0 , Actual:  0.0
Prediction:  1.0 , Actual:  1.0
Prediction:  0.0 , Actual:  4.0
Prediction:  2.0 , Actual:  6.0
Prediction:  3.0 , Actual:  3.0


In [20]:
print("correctly classified ones after training: ",correct_ones)
print("total number of instances in the test set: ",instances)

2377
3589


# Confusion Matrix

In [41]:
from sklearn.metrics import classification_report, confusion_matrix

In [42]:
confusion_matrix(y_test, predictions)

array([[233,   9,  49,  23,  90,  11,  52],
       [ 12,  31,   1,   2,   5,   0,   5],
       [ 30,   3, 189,  21, 130,  41,  82],
       [  7,   1,  11, 778,  22,  17,  59],
       [ 35,   4,  51,  24, 391,   9, 139],
       [  8,   1,  20,  18,  13, 333,  22],
       [ 13,   1,  29,  49,  90,   3, 422]])

# Exporting

In [29]:
model.export_autokeras_model('autokeras_model.h5')

In [30]:
model.export_keras_model('keras_model.h5')

# Auto-Keras Model Restoration

In [31]:
from autokeras.utils import pickle_from_file

In [32]:
autokeras_model = pickle_from_file("autokeras_model.h5")

In [33]:
autokeras_score = autokeras_model.evaluate(x_test, y_test)
print(autokeras_score)

0.6623014767344664


In [34]:
autokeras_predictions = autokeras_model.predict(x_test)

In [35]:
instances = 0
correct_ones = 0

for i in range(0, len(autokeras_predictions)):
    if i < 5:
        print("Prediction: ",autokeras_predictions[i],", Actual: ",y_test[i])
    
    if autokeras_predictions[i] == y_test[i]:
        correct_ones = correct_ones + 1
    
    instances = instances + 1

Prediction:  2.0 , Actual:  0.0
Prediction:  1.0 , Actual:  1.0
Prediction:  0.0 , Actual:  4.0
Prediction:  2.0 , Actual:  6.0
Prediction:  3.0 , Actual:  3.0


In [39]:
#this must be equal to autokeras_score
autokeras_score == correct_ones/instances

0.6623014767344664

# Keras Model Restoration

In [3]:
from keras.models import load_model

In [4]:
keras_model = load_model('keras_model.h5') 

/workspaces/sefik/.local/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
model_config = keras_model.to_json()
open("fer_autokeras_keras_model.json", "w").write(model_config)

226654

In [46]:
keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_125 (Conv2D)             (None, 48, 48, 64)   3200        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_122 (BatchN (None, 48, 48, 64)   256         conv2d_125[0][0]                 
__________________________________________________________________________________________________
activation_126 (Activation)     (None, 48, 48, 64)   0           batch_normalization_122[0][0]    
__________________________________________________________________________________________________
max_poolin

In [47]:
keras_predictions = keras_model.predict(x_test)

In [48]:
keras_predictions_exact = []

for i in range(0, len(keras_predictions)):
    keras_predictions_exact.append(np.argmax(keras_predictions[i]))

In [51]:
correctly_classified = 0

for i in range(0, keras_predictions.shape[0]):
    if i < 5:
        print("actual: ",y_test[i]," - prediction: " ,np.argmax(keras_predictions[i]))#,"(",keras_predictions[i],")")
    
    if y_test[i] == np.argmax(keras_predictions[i]):
        correctly_classified = correctly_classified + 1

actual:  0.0  - prediction:  6
actual:  1.0  - prediction:  4
actual:  4.0  - prediction:  4
actual:  6.0  - prediction:  6
actual:  3.0  - prediction:  4


In [53]:
#this must be equal to autokeras_score but this is not! it seems exporting keras wouldn't export weights. It must be retrained.
correctly_classified/predictions.shape[0] == autokeras_score

False